# Transactions Analysis

In [1]:
import time
start_time = time.time()

## 1. Set a time period

In [2]:
start_date = '2023-10-10'
end_date = '2023-10-28'

## 2. Add Input Link

In [3]:
input = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Transactions_template/Transactions analisys.xlsx"

## 3. Add Output Link

In [4]:
output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/Transactions_analisys.xlsx"

## 4. Wait for the program..

### Importing Libraries

In [5]:
import pandas as pd
import pyodbc
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

import win32com.client

### SQL Querries Loading

In [6]:
server = "scorpio"
database = "BIsmartWCBG"
conn = pyodbc.connect(f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes")

In [7]:
sql_query = f"""

DECLARE @StartDate DATE = '{start_date}';
DECLARE @EndDate DATE = '{end_date}';
 
 SELECT
ContractNumber,
--, OfferSK,
 CONVERT(DATE, CDate,4) AS [Date]
,CASE 
WHEN Name = 'A1 - 8.5%'
THEN 'A1'
WHEN Name = 'AXI 2-500'
THEN 'AXI 2'
WHEN Name = 'Visa Free ATM World'
THEN 'WhiteCard'
WHEN Name = 'Visa Бяла карта'
THEN 'WhiteCard'
WHEN Name = 'Бяла Карта'
THEN 'WhiteCard'
WHEN Name = 'ИАМ'
THEN 'WhiteCard'
WHEN Name = '180 дни Visa Бяла Карта'
THEN 'WhiteCard'
WHEN Name = 'Аванс БГ Пощи'
THEN 'WhiteCard'
WHEN Name = 'Бяла Карта – само лихва'
THEN 'WhiteCard only interest'
WHEN Name = 'Visa Бяла Карта - SC'
THEN 'WhiteCard'
WHEN Name = 'Бяла Карта Gold - 5%'
THEN 'WhiteCard-Gold'
WHEN Name = 'Бяла Карта Gold - 10%'
THEN 'WhiteCard-Gold'
WHEN Name = 'Бяла Карта Gold-MFG-5%'
THEN 'WhiteCard-Gold'
WHEN Name = 'Кредит от БК 3'
THEN 'Axi Credit 3'
WHEN Name = 'Кредит от БК 4 '
THEN 'Axi Credit 4'
WHEN Name = 'Кредит от БК 4 - paper'
THEN 'Axi Credit 4'
WHEN Name = 'Бяла Карта 2'
THEN 'WhiteCard'
ELSE Name
END AS Product

,CASE 
WHEN TransactionChanel = 'POSCASH'
THEN 'POS'
WHEN TransactionChanel = 'MANUAL'
THEN 'POS'
ELSE TransactionChanel
END AS TransactionChanel

,CASE
When Name in ('AXI 2', 'AXI 2-500', 'Бяла Карта Gold - 5%', 'Бяла Карта Gold - 10%' ) and TransactionChanel = 'ATM'
then SUM(ATMAmount) 
Else  SUM(AmountTransactionBilling * -1) 
END AS AmountReal

, SUM(AmountTransactionBilling * -1) AS Amount
, COUNT(AmountTransactionBilling) AS CountTransaction
, SUM(ATMAmount) AS AxiATM
 FROM(
SELECT --fct.OfferSK
 ContractNumber
, fct.CDate
, p.Name
, AmountTransactionBilling
, TransactionDate
, tc.Name AS TransactionChanel,
 ((-AmountTransactionBilling - 1.5)/1.03) AS ATMAmount
from dwh.FactCardTransactions fct
   JOIN dwh.DimOffers do ON do.OfferSK = fct.OfferSK
   JOIN DWh.DimProduct p ON p.ProductSK = do.ProductSK
   JOIN dwh.DimTransactionChannels tc ON tc.TransactionChannelSK = fct.TransactionChannelSK
   WHERE tc.Name NOT IN ('ANNUAL_TAX', 'BALANCE', 'PIN_CHANGE', 'FEE')
   AND ResponseCodeSK = 47
   AND AmountTransactionBilling < 0 
   AND CONVERT(DATE, fct.CDate, 4) >= @StartDate
   AND CONVERT(DATE, fct.CDate, 4) <= @EndDate

   ) AS Result
     GROUP BY Name, TransactionChanel, CONVERT(DATE, CDate,4), ContractNumber -- OfferSK
ORDER BY Date, Name

"""

In [8]:
data_sql = pd.read_sql_query(sql_query, conn)
data_sql

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_8728\2272456572.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_sql = pd.read_sql_query(sql_query, conn)


,ContractNumber,Date,Product,TransactionChanel,AmountReal,Amount,CountTransaction,AxiATM
0,554324,2023-10-10,A1,ATM,80.00,80.00,1,76.213592
1,555258,2023-10-10,A1,ATM,10.00,10.00,1,8.252427
2,555407,2023-10-10,A1,ATM,100.00,100.00,1,95.631068
3,555412,2023-10-10,A1,ATM,400.00,400.00,1,386.893204
4,555440,2023-10-10,A1,ATM,100.00,100.00,1,95.631068
...,...,...,...,...,...,...,...,...
112932,409945,2023-10-28,WhiteCard,POS,51.50,51.50,1,48.543689
112933,410909,2023-10-28,WhiteCard,POS,9.30,9.30,1,7.572816
112934,411528,2023-10-28,WhiteCard,POS,40.99,40.99,2,36.883495
112935,411539,2023-10-28,WhiteCard,POS,1.60,1.60,1,0.097087


### Excel file Loading

In [9]:
start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d.%m')
end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%d.%m')
period = start_date + "-" + end_date

In [10]:
existing_data = pd.read_excel(input)

In [11]:
columns_to_drop = [f"Общо брой трансакции_{period}",
                 f"Обща сума на всички направени трансакции_{period}",
                 f"Брой трансакции на ПОС {period}",
                 f"Брой трансакции на АТМ {period}",
                 f"Брой трансакции онлайн {period}"]
columns_to_drop

['Общо брой трансакции_10.10-28.10',
 'Обща сума на всички направени трансакции_10.10-28.10',
 'Брой трансакции на ПОС 10.10-28.10',
 'Брой трансакции на АТМ 10.10-28.10',
 'Брой трансакции онлайн 10.10-28.10']

In [12]:
existing_data.drop(columns=columns_to_drop, inplace=True)

In [13]:
workbook = load_workbook(input)

In [14]:
worksheet = workbook.active
worksheet

<Worksheet "Sheet1">

In [15]:
total_count = data_sql.pivot_table(index="ContractNumber",
                                  values="CountTransaction",
                                  aggfunc="sum")
total_count.rename(columns={"CountTransaction": f"Общо брой трансакции_{period}"}, inplace=True)
total_count.applymap(lambda x: 0 if pd.isna(x) else x)

,Общо брой трансакции_10.10-28.10
ContractNumber,
400025,4
400028,1
400041,4
400045,5
400078,1
...,...
11073212,2
11073227,1
11073228,1


In [16]:
total_amount = data_sql.pivot_table(index="ContractNumber",
                                  values="AmountReal",
                                  aggfunc="sum")
total_amount.rename(columns={"AmountReal": f"Обща сума на всички направени трансакции_{period}"}, inplace=True)
total_amount.applymap(lambda x: 0 if pd.isna(x) else x)

,Обща сума на всички направени трансакции_10.10-28.10
ContractNumber,
400025,6.40
400028,200.00
400041,700.00
400045,172.93
400078,400.00
...,...
11073212,150.00
11073227,400.00
11073228,200.00


In [17]:
new_columns = [f"Брой трансакции на ПОС {period}", f"Брой трансакции на ATM {period}", f"Брой трансакции онлайн {period}"]

transactions_by_channels = data_sql.pivot_table(index='ContractNumber',
                                                columns='TransactionChanel',
                                                values='CountTransaction',
                                                aggfunc='sum',
                                                fill_value=0).reset_index()

transactions_by_channels.rename(columns={
    'POS': new_columns[0],
    'ATM': new_columns[1],
    'ECOMM': new_columns[2]
}, inplace=True)


desired_order = [
    new_columns[0],
    new_columns[1],
    new_columns[2]
]
transactions_by_channels = transactions_by_channels[desired_order + ['ContractNumber']]
transactions_by_channels.fillna(0, inplace=True)

### Merged total_count, total_amount and transactions_by_channels 

In [18]:
merged_1 = total_count.merge(total_amount, on='ContractNumber', how='inner')
result = merged_1.merge(transactions_by_channels, on='ContractNumber', how='inner')
result

,ContractNumber,Общо брой трансакции_10.10-28.10,Обща сума на всички направени трансакции_10.10-28.10,Брой трансакции на ПОС 10.10-28.10,Брой трансакции на ATM 10.10-28.10,Брой трансакции онлайн 10.10-28.10
0,400025,4,6.40,4,0,0
1,400028,1,200.00,0,1,0
2,400041,4,700.00,1,3,0
3,400045,5,172.93,5,0,0
4,400078,1,400.00,0,1,0
...,...,...,...,...,...,...
34105,11073212,2,150.00,0,2,0
34106,11073227,1,400.00,0,1,0
34107,11073228,1,200.00,0,1,0
34108,11073238,1,300.00,0,1,0


In [19]:
data = existing_data.merge(result, left_on="EasyClientNumber", right_on="ContractNumber", how="left")
data.drop(columns="ContractNumber", inplace=True)

In [20]:
valid_columns = [
    'Текущ издател',
    'Продукт',
    'EasyClientNumber',
    'Лимит',
    'Клиент',
    'ЕГН',
    'Телефон',
    'Статус на картата',
    'Дата на първа активация на картата',
    'Дължимо до зануляване',
    'Разполагаема сума',
    'Дни забава',
    'Marketig_consent',
    'SMS_Consent',
    'Email_Consent',
    'Email',
    'Дата на подписване на договор',
    'Дата на последна трансакция'
]

for column in data.columns:
    if column not in valid_columns:
        data[column].fillna(0, inplace=True)
    else:
        data[column].fillna("#N/A", inplace=True)    

### Data Export

In [21]:
data_rows = list(dataframe_to_rows(data, index=False, header=False))

In [22]:
start_row = 2 
start_column = 1

In [23]:
for row in data_rows:
    for value in row:
        worksheet.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1

In [24]:
workbook.save(output)

### Email Anouncing

In [25]:
recipient_email = "alexi.zein@gmail.com"
subject = "Transaction Analisys"
message = """Dear colleagues,

This email is automatically generated and contains information about added data.

Date and time of generation: {}

The Transactions Analisys data has been added successfully and can be found in the shared folder on PD.

""".format(datetime.now().strftime("%d %B %Y, %H:%M ч."))

In [26]:
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

In [27]:
mail = outlook.CreateItem(0)
mail.Subject = subject
mail.Body = message
mail.To = recipient_email

In [28]:
mail.Send()

In [29]:
print("The notification email has been successfully sent.")

The notification email has been successfully sent.


In [30]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"The script executed for {execution_time_minutes:.2f} minutes.")

The script executed for 0.60 minutes.
